In [1]:
'''
(C) Renata Wong 2023


Note: Below we perform calculations on the molecular Hamiltonian of H_2.
To perform calculations on other molecules, you will need to specify their geometry, charge and spin 
to replace the values in the driver. 

To do: Hamiltonian for H_2 on 8 qubits using 6-31G basis.
'''

import numpy as np
import matplotlib.pyplot as plt
import time

from functools import partial

from qiskit.circuit.library import EfficientSU2

from qiskit_aer import QasmSimulator
from qiskit import QuantumCircuit, execute

from qiskit.algorithms.optimizers import SLSQP, COBYLA, SPSA

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

from collections import Counter

from qiskit_nature.second_q.mappers import BravyiKitaevMapper, QubitConverter
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit.opflow import I, StateFn, CircuitStateFn

from predicting_quantum_properties.data_acquisition_shadow import derandomized_classical_shadow
from predicting_quantum_properties.prediction_shadow import estimate_exp

from modified_derandomization import modified_derandomized_classical_shadow





# SPECIFY THE NUMBER OF EXPERIMENTS YOU WANT TO RUN
num_experiments = 10

# SPECIFY THE EXPECTED GROUND STATE ENERGY FOR THE MOLECULE OF INTEREST
EXPECTED_EIGENVALUE = -1.86



# SPECIFY THE GEOMETRY OF THE MOLECULE IN QUESTION
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)


problem = driver.run()
hamiltonian = problem.hamiltonian

# The electronic Hamiltonian of the system
second_q_op = hamiltonian.second_q_op()

# Solving the electronic structure problem = determine the ground state energy of the molecule
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, NumPyMinimumEigensolverFactory
from qiskit_nature.second_q.mappers import BravyiKitaevMapper

# The Bravyi-Kitaev repserentation of the Fermionic Hamiltonian
mapper = BravyiKitaevMapper()
bkenc_hamiltonian = mapper.map(second_q_op)




'''
Format Hamiltonian terms and coefficients as required by the package predicting-quantum-properties. 
This includes removing all Pauli-I operators.
'''

hamiltonian_terms = []
weights = []

for observable in bkenc_hamiltonian:
    
    observable_str = str(observable)
    observable_str_clean = observable_str.strip()  # removes white spaces
    pauli_str_list = observable_str_clean.split('*')
    tuple_list = []
    
    for op_index, pauli_op in enumerate(pauli_str_list[1]):
        if pauli_op == 'I' or pauli_op == 'X' or pauli_op == 'Y' or pauli_op == 'Z':
            tuple_list.append((pauli_op, op_index-1))
    if len(tuple_list) > 0:
        hamiltonian_terms.append(tuple_list)
        weights.append(float(pauli_str_list[0].strip()))

system_size = len(hamiltonian_terms[0])





hamiltonian_terms_XYZE = []

for term in hamiltonian_terms:
    term_XYZE = []
    for pauli in term:
        if pauli[0] != 'I':
            term_XYZE.append(pauli)
    hamiltonian_terms_XYZE.append(term_XYZE)   
    

weights_XYZ = weights.copy()
weights_XYZ.pop(0)

abs_weights = [abs(weight) for weight in weights]

hamiltonian_terms_XYZ = []
for idx, term in enumerate(hamiltonian_terms_XYZE):
    if term:
        hamiltonian_terms_XYZ.append(term)
    else:
        abs_weights.pop(idx)
        

/home/renata/.local/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'
/home/renata/.local/lib/python3.8/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


In [2]:
'''
DIVERSITY IN DERANDOMIZED OPERATORS USING MODIFIED DERANDOMIZATION, COMPARED TO 
DIVERSITY IN DERANDOMIZED OPERATORS USING ORIGINAL DERANDOMIZATION PROCEDURE.
'''

for num_operators in range(1, 5001, 50):
    
    if num_operators == 1:
        num_operators = 2
        
    derandomized_hamiltonian = modified_derandomized_classical_shadow(hamiltonian_terms_XYZ, 
                                                                      num_operators-1, system_size, weight=abs_weights)

    tuples = (tuple(pauli) for pauli in derandomized_hamiltonian)
    counts = Counter(tuples)
    print('MOD {} | {}'.format(num_operators-1, counts))




MOD 1 | Counter({('X', 'X', 'X', 'X'): 1})
MOD 50 | Counter({('X', 'X', 'X', 'X'): 50})
MOD 100 | Counter({('X', 'X', 'X', 'X'): 99, ('Z', 'Z', 'Z', 'Z'): 1})
MOD 150 | Counter({('X', 'X', 'X', 'X'): 148, ('Z', 'Z', 'Z', 'Z'): 2})
MOD 200 | Counter({('X', 'X', 'X', 'X'): 197, ('Z', 'Z', 'Z', 'Z'): 3})
MOD 250 | Counter({('X', 'X', 'X', 'X'): 246, ('Z', 'Z', 'Z', 'Z'): 4})
MOD 300 | Counter({('X', 'X', 'X', 'X'): 296, ('Z', 'Z', 'Z', 'Z'): 4})
MOD 350 | Counter({('X', 'X', 'X', 'X'): 344, ('Z', 'Z', 'Z', 'Z'): 5, ('Z', 'X', 'Z', 'X'): 1})
MOD 400 | Counter({('X', 'X', 'X', 'X'): 393, ('Z', 'Z', 'Z', 'Z'): 6, ('Z', 'X', 'Z', 'X'): 1})
MOD 450 | Counter({('X', 'X', 'X', 'X'): 442, ('Z', 'Z', 'Z', 'Z'): 7, ('Z', 'X', 'Z', 'X'): 1})
MOD 500 | Counter({('X', 'X', 'X', 'X'): 491, ('Z', 'Z', 'Z', 'Z'): 8, ('Z', 'X', 'Z', 'X'): 1})
MOD 550 | Counter({('X', 'X', 'X', 'X'): 541, ('Z', 'Z', 'Z', 'Z'): 8, ('Z', 'X', 'Z', 'X'): 1})
MOD 600 | Counter({('X', 'X', 'X', 'X'): 590, ('Z', 'Z', 'Z', 'Z'): 

MOD 3250 | Counter({('X', 'X', 'X', 'X'): 3187, ('Z', 'Z', 'Z', 'Z'): 51, ('Z', 'X', 'Z', 'X'): 10, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
MOD 3300 | Counter({('X', 'X', 'X', 'X'): 3236, ('Z', 'Z', 'Z', 'Z'): 52, ('Z', 'X', 'Z', 'X'): 10, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
MOD 3350 | Counter({('X', 'X', 'X', 'X'): 3285, ('Z', 'Z', 'Z', 'Z'): 53, ('Z', 'X', 'Z', 'X'): 10, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
MOD 3400 | Counter({('X', 'X', 'X', 'X'): 3335, ('Z', 'Z', 'Z', 'Z'): 53, ('Z', 'X', 'Z', 'X'): 10, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
MOD 3450 | Counter({('X', 'X', 'X', 'X'): 3383, ('Z', 'Z', 'Z', 'Z'): 54, ('Z', 'X', 'Z', 'X'): 11, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
MOD 3500 | Counter({('X', 'X', 'X', 'X'): 3432, ('Z', 'Z', 'Z', 'Z'): 55, ('Z', 'X', 'Z', 'X'): 11, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
MOD 3550 | Counter({('X', 'X', 'X', 'X'): 3481, ('Z', 'Z', 'Z', 'Z'): 56, ('Z', 'X', 'Z', 'X')

In [3]:
for num_evals in range(1, 1001):

              
    derandomized_hamiltonian = derandomized_classical_shadow(hamiltonian_terms_XYZ, 
                                                                    num_evals, system_size, weight=abs_weights)
     
    tuples = (tuple(pauli) for pauli in derandomized_hamiltonian)
    counts = Counter(tuples)
    print('ORG {} | {}'.format(num_evals, counts))


ORG 1 | Counter({('X', 'X', 'X', 'X'): 1})
ORG 2 | Counter({('X', 'X', 'X', 'X'): 1})
ORG 3 | Counter({('X', 'X', 'X', 'X'): 1})
ORG 4 | Counter({('X', 'X', 'X', 'X'): 1})
ORG 5 | Counter({('Z', 'Z', 'Z', 'Z'): 1})
ORG 6 | Counter({('Z', 'Z', 'Z', 'Z'): 1})
ORG 7 | Counter({('Z', 'Z', 'Z', 'Z'): 1})
ORG 8 | Counter({('Z', 'Z', 'Z', 'Z'): 1})
ORG 9 | Counter({('Z', 'Z', 'Z', 'Z'): 2})
ORG 10 | Counter({('Z', 'Z', 'Z', 'Z'): 2})
ORG 11 | Counter({('Z', 'Z', 'Z', 'Z'): 2})
ORG 12 | Counter({('Z', 'Z', 'Z', 'Z'): 2})
ORG 13 | Counter({('Z', 'Z', 'Z', 'Z'): 2})
ORG 14 | Counter({('Z', 'Z', 'Z', 'Z'): 3})
ORG 15 | Counter({('Z', 'Z', 'Z', 'Z'): 3})
ORG 16 | Counter({('Z', 'Z', 'Z', 'Z'): 3})
ORG 17 | Counter({('Z', 'Z', 'Z', 'Z'): 3})
ORG 18 | Counter({('Z', 'Z', 'Z', 'Z'): 4})
ORG 19 | Counter({('Z', 'Z', 'Z', 'Z'): 4})
ORG 20 | Counter({('Z', 'Z', 'Z', 'Z'): 4})
ORG 21 | Counter({('Z', 'Z', 'Z', 'Z'): 4})
ORG 22 | Counter({('Z', 'Z', 'Z', 'Z'): 4})
ORG 23 | Counter({('Z', 'Z', 'Z', 'Z'): 5

ORG 125 | Counter({('Z', 'Z', 'Z', 'Z'): 27, ('Z', 'X', 'Z', 'X'): 5, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 126 | Counter({('Z', 'Z', 'Z', 'Z'): 27, ('Z', 'X', 'Z', 'X'): 5, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 127 | Counter({('Z', 'Z', 'Z', 'Z'): 27, ('Z', 'X', 'Z', 'X'): 5, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 128 | Counter({('Z', 'Z', 'Z', 'Z'): 27, ('Z', 'X', 'Z', 'X'): 5, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 129 | Counter({('Z', 'Z', 'Z', 'Z'): 28, ('Z', 'X', 'Z', 'X'): 5, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 130 | Counter({('Z', 'Z', 'Z', 'Z'): 28, ('Z', 'X', 'Z', 'X'): 5, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 131 | Counter({('Z', 'Z', 'Z', 'Z'): 28, ('Z', 'X', 'Z', 'X'): 5, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 132 | Counter({('Z', 'Z', 'Z', 'Z'): 28, ('Z', 'X', 'Z', 'X'): 5, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 133 | Counter({('Z', 'Z', 'Z

ORG 202 | Counter({('Z', 'Z', 'Z', 'Z'): 44, ('Z', 'X', 'Z', 'X'): 9, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 203 | Counter({('Z', 'Z', 'Z', 'Z'): 44, ('Z', 'X', 'Z', 'X'): 9, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 204 | Counter({('Z', 'Z', 'Z', 'Z'): 45, ('Z', 'X', 'Z', 'X'): 9, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 205 | Counter({('Z', 'Z', 'Z', 'Z'): 45, ('Z', 'X', 'Z', 'X'): 9, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 206 | Counter({('Z', 'Z', 'Z', 'Z'): 45, ('Z', 'X', 'Z', 'X'): 9, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 207 | Counter({('Z', 'Z', 'Z', 'Z'): 45, ('Z', 'X', 'Z', 'X'): 9, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 208 | Counter({('Z', 'Z', 'Z', 'Z'): 45, ('Z', 'X', 'Z', 'X'): 9, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 209 | Counter({('Z', 'Z', 'Z', 'Z'): 46, ('Z', 'X', 'Z', 'X'): 9, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 210 | Counter({('Z', 'Z', 'Z

ORG 273 | Counter({('Z', 'Z', 'Z', 'Z'): 60, ('Z', 'X', 'Z', 'X'): 12, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 274 | Counter({('Z', 'Z', 'Z', 'Z'): 60, ('Z', 'X', 'Z', 'X'): 12, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 275 | Counter({('Z', 'Z', 'Z', 'Z'): 61, ('Z', 'X', 'Z', 'X'): 12, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 276 | Counter({('Z', 'Z', 'Z', 'Z'): 61, ('Z', 'X', 'Z', 'X'): 12, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 277 | Counter({('Z', 'Z', 'Z', 'Z'): 61, ('Z', 'X', 'Z', 'X'): 12, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 278 | Counter({('Z', 'Z', 'Z', 'Z'): 61, ('Z', 'X', 'Z', 'X'): 12, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 279 | Counter({('Z', 'Z', 'Z', 'Z'): 61, ('Z', 'X', 'Z', 'X'): 12, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 280 | Counter({('Z', 'Z', 'Z', 'Z'): 62, ('Z', 'X', 'Z', 'X'): 12, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 281 | Counter({('Z',

ORG 343 | Counter({('Z', 'Z', 'Z', 'Z'): 76, ('Z', 'X', 'Z', 'X'): 15, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 344 | Counter({('Z', 'Z', 'Z', 'Z'): 76, ('Z', 'X', 'Z', 'X'): 15, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 345 | Counter({('Z', 'Z', 'Z', 'Z'): 76, ('Z', 'X', 'Z', 'X'): 15, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 346 | Counter({('Z', 'Z', 'Z', 'Z'): 77, ('Z', 'X', 'Z', 'X'): 15, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 347 | Counter({('Z', 'Z', 'Z', 'Z'): 77, ('Z', 'X', 'Z', 'X'): 15, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 348 | Counter({('Z', 'Z', 'Z', 'Z'): 77, ('Z', 'X', 'Z', 'X'): 15, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 349 | Counter({('Z', 'Z', 'Z', 'Z'): 77, ('Z', 'X', 'Z', 'X'): 15, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 350 | Counter({('Z', 'Z', 'Z', 'Z'): 78, ('Z', 'X', 'Z', 'X'): 15, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 351 | Counter({('Z',

...

ORG 691 | Counter({('Z', 'Z', 'Z', 'Z'): 154, ('Z', 'X', 'Z', 'X'): 31, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 692 | Counter({('Z', 'Z', 'Z', 'Z'): 155, ('Z', 'X', 'Z', 'X'): 31, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 693 | Counter({('Z', 'Z', 'Z', 'Z'): 155, ('Z', 'X', 'Z', 'X'): 31, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 694 | Counter({('Z', 'Z', 'Z', 'Z'): 155, ('Z', 'X', 'Z', 'X'): 31, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 695 | Counter({('Z', 'Z', 'Z', 'Z'): 155, ('Z', 'X', 'Z', 'X'): 31, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 696 | Counter({('Z', 'Z', 'Z', 'Z'): 156, ('Z', 'X', 'Z', 'X'): 31, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 697 | Counter({('Z', 'Z', 'Z', 'Z'): 156, ('Z', 'X', 'Z', 'X'): 31, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 698 | Counter({('Z', 'Z', 'Z', 'Z'): 156, ('Z', 'X', 'Z', 'X'): 31, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 699 | Counte

ORG 760 | Counter({('Z', 'Z', 'Z', 'Z'): 170, ('Z', 'X', 'Z', 'X'): 34, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 761 | Counter({('Z', 'Z', 'Z', 'Z'): 170, ('Z', 'X', 'Z', 'X'): 34, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 762 | Counter({('Z', 'Z', 'Z', 'Z'): 171, ('Z', 'X', 'Z', 'X'): 34, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 763 | Counter({('Z', 'Z', 'Z', 'Z'): 171, ('Z', 'X', 'Z', 'X'): 34, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 764 | Counter({('Z', 'Z', 'Z', 'Z'): 171, ('Z', 'X', 'Z', 'X'): 34, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 765 | Counter({('Z', 'Z', 'Z', 'Z'): 171, ('Z', 'X', 'Z', 'X'): 34, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 766 | Counter({('Z', 'Z', 'Z', 'Z'): 171, ('Z', 'X', 'Z', 'X'): 34, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 767 | Counter({('Z', 'Z', 'Z', 'Z'): 172, ('Z', 'X', 'Z', 'X'): 34, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 768 | Counte

ORG 829 | Counter({('Z', 'Z', 'Z', 'Z'): 186, ('Z', 'X', 'Z', 'X'): 37, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 830 | Counter({('Z', 'Z', 'Z', 'Z'): 186, ('Z', 'X', 'Z', 'X'): 37, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 831 | Counter({('Z', 'Z', 'Z', 'Z'): 186, ('Z', 'X', 'Z', 'X'): 37, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 832 | Counter({('Z', 'Z', 'Z', 'Z'): 186, ('Z', 'X', 'Z', 'X'): 37, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 833 | Counter({('Z', 'Z', 'Z', 'Z'): 187, ('Z', 'X', 'Z', 'X'): 37, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 834 | Counter({('Z', 'Z', 'Z', 'Z'): 187, ('Z', 'X', 'Z', 'X'): 37, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 835 | Counter({('Z', 'Z', 'Z', 'Z'): 187, ('Z', 'X', 'Z', 'X'): 37, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 836 | Counter({('Z', 'Z', 'Z', 'Z'): 187, ('Z', 'X', 'Z', 'X'): 37, ('Z', 'X', 'Z', 'Z'): 1, ('Z', 'Z', 'Z', 'X'): 1})
ORG 837 | Counte

ORG 889 | Counter({('X', 'X', 'X', 'X'): 12222, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 890 | Counter({('X', 'X', 'X', 'X'): 12236, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 891 | Counter({('X', 'X', 'X', 'X'): 12250, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 892 | Counter({('X', 'X', 'X', 'X'): 12264, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 893 | Counter({('X', 'X', 'X', 'X'): 12278, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 894 | Counter({('X', 'X', 'X', 'X'): 12292, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 895 | Counter({('X', 'X', 'X', 'X'): 12306, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 896 | Counter({('X', 'X', 'X', 'X'): 12320, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X',

ORG 954 | Counter({('X', 'X', 'X', 'X'): 13132, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 955 | Counter({('X', 'X', 'X', 'X'): 13146, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 956 | Counter({('X', 'X', 'X', 'X'): 13160, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 957 | Counter({('X', 'X', 'X', 'X'): 13174, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 958 | Counter({('X', 'X', 'X', 'X'): 13188, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 959 | Counter({('X', 'X', 'X', 'X'): 13202, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 960 | Counter({('X', 'X', 'X', 'X'): 13216, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X', 'Z', 'Z'): 1})
ORG 961 | Counter({('X', 'X', 'X', 'X'): 13230, ('Z', 'Z', 'Z', 'Z'): 185, ('Z', 'X', 'Z', 'X'): 38, ('Z', 'X',